# VTK - ITK Project : Etude longitudinale de l'évolution d'une tumeur

### Par Raphaël Duhen, Maël Conan et Nigel Andrews

# Imports

In [265]:
from typing import Tuple, Any

import vtk
import itk

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output

# Fonction *helper* permettant de rendre un fichier .nrrd en 3D issue des TPs.

In [266]:
def run_window(path: str):
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(path)
    
    window = vtk.vtkRenderWindow()
    renderer = vtk.vtkRenderer()
    
    window.AddRenderer(renderer)
    
    interactor = vtk.vtkRenderWindowInteractor()
    window.SetInteractor(interactor)
    
    contour = vtk.vtkContourFilter()
    contour.SetInputConnection(reader.GetOutputPort())
    contour.SetValue(0, 135)
    
    contourMapper = vtk.vtkPolyDataMapper()
    contourMapper.SetInputConnection(contour.GetOutputPort())
    contourMapper.ScalarVisibilityOff()
    
    contourActor = vtk.vtkActor()
    contourActor.SetMapper(contourMapper)
    
    renderer.AddActor(contourActor)
    
    window.Render()
    interactor.Start()

# Constantes

Le type de pixel et les dimensions de l'image n'étant pas sujet à changements dans notre *pipeline*, nous les définissons globalement ici.

In [267]:
PixelType = itk.D
Dimension = 3
ImageType = itk.Image[PixelType, Dimension]
TransformType = itk.TranslationTransform[PixelType, Dimension]

# 2. Recalage d'images

In [268]:
image = itk.imread("Data/case6_gre1.nrrd", pixel_type=PixelType)
image2 = itk.imread("Data/case6_gre2.nrrd", pixel_type=PixelType) 

On effectue un très léger lissage des images pour diminuer le bruit et faciliter le recalage.

In [269]:
image = itk.median_image_filter(image, radius=1)
image2 = itk.median_image_filter(image2, radius=1)

resampler = itk.ResampleImageFilter[ImageType, ImageType].New()
resampler.SetInput(image)
resampler.SetSize(image.GetLargestPossibleRegion().GetSize())
resampler.SetOutputSpacing(image.GetSpacing())
resampler.SetOutputOrigin(image.GetOrigin())
resampler.SetOutputDirection(image.GetDirection())
resampler.Update()

resampler2 = itk.ResampleImageFilter[ImageType, ImageType].New()
resampler2.SetInput(image2)
resampler2.SetSize(image2.GetLargestPossibleRegion().GetSize())
resampler2.SetOutputSpacing(image2.GetSpacing())
resampler2.SetOutputOrigin(image2.GetOrigin())
resampler2.SetOutputDirection(image2.GetDirection())
resampler2.Update()

image = resampler.GetOutput()
image2 = resampler2.GetOutput()

In [270]:
def registrate(image, image2):
    dimension = image.GetImageDimension()
    TransformType = itk.TranslationTransform[itk.D, dimension]

    initial_transform = TransformType.New()
    initial_transform.SetIdentity()

    optimizer = itk.RegularStepGradientDescentOptimizerv4.New(
        LearningRate=4,
        MinimumStepLength=0.001,
        RelaxationFactor=0.5,
        NumberOfIterations=200,
    )

    metric = itk.MeanSquaresImageToImageMetricv4[ImageType, ImageType].New()

    registration = itk.ImageRegistrationMethodv4[ImageType, ImageType].New(
        Metric=metric,
        Optimizer=optimizer,
        InitialTransform=initial_transform,
        FixedImage=image,
        MovingImage=image2,
    )

    registration.Update()

    return registration, optimizer

In [271]:
def transform_image(image, image2, registration):
    transformed_image = itk.resample_image_filter(
        image2,
        transform=registration.GetTransform(),
        use_reference_image=True,
        reference_image=image,
        default_pixel_value=100,
    )

    return transformed_image

In [272]:
registration = None

try:
    transformed_image = itk.imread("Data/transformed_image.nrrd", pixel_type=PixelType)
except:
    registration, optimizer = registrate(image, image2)
    transformed_image = transform_image(image, image2, registration)
    itk.imwrite(transformed_image, "Data/transformed_image.nrrd")

Visualisation interactive pour vérifier les résultats du recalage.

In [273]:
clear_output()

array_view = itk.array_view_from_image(transformed_image)
original_fixed_view = itk.array_view_from_image(image)
original_moving_view = itk.array_view_from_image(image2)

@interact(fixed=(0, original_fixed_view.shape[0]),
          moving=(0, original_moving_view.shape[0]),
          transformed=(0, array_view.shape[0]))
def plot_slices(fixed: int = 81, moving: int = 81, transformed: int = 81):
    plt.figure(figsize=(15, 15))

    plt.subplot(1, 3, 1)
    plt.imshow(original_fixed_view[fixed, :, :], cmap="gray", interpolation_stage='rgba')
    plt.title("Fixed image")

    plt.subplot(1, 3, 2)
    plt.imshow(original_moving_view[moving, :, :], cmap="gray", interpolation_stage='rgba')
    plt.title("Moving image")

    plt.subplot(1, 3, 3)
    plt.imshow(array_view[transformed, :, :], cmap="gray", interpolation_stage='rgba')
    plt.title("Transformed image")

    plt.show()

interactive(children=(IntSlider(value=81, description='fixed', max=176), IntSlider(value=81, description='movi…

In [274]:
if registration is not None:
    final_parameters = registration.GetOutput().Get().GetParameters()

    final_number_of_iterations = optimizer.GetCurrentIteration()

    print("Final parameters = ", final_parameters.GetElement(0))
    print("Final parameters = ", final_parameters.GetElement(1))
    print("Final parameters = ", final_parameters.GetElement(2))
    print("Final parameters = ", final_parameters.GetElement(3))
    print("Final parameters = ", final_parameters.GetElement(4))
    print("Final parameters = ", final_parameters.GetElement(5))

    print("Number of iterations = ", final_number_of_iterations)

# 3. Segmentation des tumeurs

In [275]:
def segmentation(image, lower: int = 400):
    PixelType = itk.D
    Dimension = 3
    ImageType = itk.Image[PixelType, Dimension]

    segmentation_filter = itk.ConnectedThresholdImageFilter[ImageType, ImageType].New(
        Lower=lower,
        Upper=1200,
        ReplaceValue=255,
        # This seed corresponds to a spot roughly near the center of the tumor.
        Seed=itk.Index[Dimension]([60, 120, 82]),
    )
    segmentation_filter.SetInput(image)
    segmentation_filter.Update()

    return segmentation_filter.GetOutput()

In [276]:
segmented_image = segmentation(transformed_image, lower=300)

In [277]:
segmented_slices = itk.array_view_from_image(segmented_image)

In [278]:
masked_slices = ma.masked_array(array_view, mask=(segmented_slices == 0))

In [279]:
mask_cmap = matplotlib.cm.inferno
mask_cmap.set_bad('black', 1)

In [280]:
clear_output()

@interact(slice=(0, segmented_slices.shape[0] - 1))
def plot_segmentation(slice: int = 81):
    plt.figure(figsize=(15, 15))

    plt.subplot(1, 3, 1)
    plt.imshow(array_view[slice, :, :], cmap="inferno", interpolation_stage='rgba')
    plt.title("Transformed image")

    plt.subplot(1, 3, 2)
    plt.imshow(segmented_slices[slice, :, :], cmap="gray", interpolation_stage='rgba')
    plt.title("Segmented image")

    plt.subplot(1, 3, 3)
    plt.imshow(masked_slices[slice, :, :], cmap=mask_cmap, interpolation_stage='rgba')
    plt.title("Mask result")
    plt.show()

interactive(children=(IntSlider(value=81, description='slice', max=175), Output()), _dom_classes=('widget-inte…

In [281]:
box = [(0, segmented_slices.shape[0]), (0, segmented_slices.shape[1]), (0, segmented_slices.shape[2])]

@interact(
    x=widgets.IntRangeSlider(value=[103,141], min=0, max=segmented_slices.shape[1]),
    y=widgets.IntRangeSlider(value=[56,77], min=0, max=segmented_slices.shape[2]),
    z=widgets.IntRangeSlider(value=[69,89], min=0, max=segmented_slices.shape[0]),
    side_slice=(0, segmented_slices.shape[0]),
    top_slice=(0, segmented_slices.shape[1]),
)
def handfind_tumor_bounds(x: Tuple[int, int], y: Tuple[int, int], z: Tuple[int, int], side_slice: int = 81, top_slice: int = 66):
    plt.figure(figsize=(15, 15))

    plt.subplot(1, 2, 1)
    plt.imshow(masked_slices[side_slice, y[0]:y[1], x[0]:x[1]], cmap=mask_cmap, interpolation_stage='rgba')
    plt.title("Side view")

    ax = plt.subplot(1, 2, 2)
    plt.imshow(masked_slices[z[0]:z[1], top_slice, x[0]:x[1]], cmap=mask_cmap, interpolation_stage='rgba')
    plt.title("Top view")

    global box
    box = [tuple([*z]), tuple([*y]), tuple([*x])]

interactive(children=(IntRangeSlider(value=(103, 141), description='x', max=256), IntRangeSlider(value=(56, 77…

In [282]:
box_slice = [slice(*tup) for tup in box]
print(box_slice)

[slice(69, 89, None), slice(56, 77, None), slice(103, 141, None)]


In [283]:
box_mask = np.zeros(masked_slices.shape, dtype=bool)
box_mask[*box_slice] = True
boxed_tumor_slices = ma.masked_array(masked_slices, ~box_mask).filled(0)

In [284]:
clear_output()

@interact(slice=box[0])
def show_mask(slice: int = 81):
    plt.imshow(boxed_tumor_slices[slice, :, : ], cmap=mask_cmap) # *box_slice[1:3]

interactive(children=(IntSlider(value=81, description='slice', max=89, min=69), Output()), _dom_classes=('widg…

In [285]:
# Uncomment to open tumor in viewer
# itk.imwrite(itk.image_view_from_array(boxed_tumor_slices), "Data/tumor2.nrrd")
# run_window("Data/tumor2.nrrd")

Avec notre résultat de segmentation satisfaisant, nous pouvons tenter de l'appliquer tel quel à l'image de départ (non transformée).

In [286]:
def run():
    image = itk.imread("Data/case6_gre1.nrrd", pixel_type=itk.D)

    image = itk.median_image_filter(image, radius=1)

    resampler = itk.ResampleImageFilter[ImageType, ImageType].New()
    resampler.SetInput(image)
    resampler.SetSize(image.GetLargestPossibleRegion().GetSize())
    resampler.SetOutputSpacing(image.GetSpacing())
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    resampler.Update()

    image = resampler.GetOutput()
    image_slices = itk.array_view_from_image(image)

    segmented = segmentation(image, lower=300)
    segmented_slices = itk.array_view_from_image(segmented) 

    masked_slices = ma.masked_array(image_slices, mask=(segmented_slices == 0))
    boxed_tumor_slices = ma.masked_array(masked_slices, ~box_mask).filled(0)

    return boxed_tumor_slices

original_slices = run()

@interact(slice=(70, 88))
def display(slice: int = 81):
    plt.imshow(original_slices[slice, *box_slice[1:3]], cmap=mask_cmap)

interactive(children=(IntSlider(value=81, description='slice', max=88, min=70), Output()), _dom_classes=('widg…

On peut voir que les tranches 87 et 88 contiennent énormément de bruit. Nous allons donc les exclure des deux images.

In [287]:
original_slices = original_slices[:87]
boxed_tumor_slices = boxed_tumor_slices[:87]

In [288]:
# Uncomment to open original tumor in viewer
# itk.imwrite(itk.image_view_from_array(original_slices), "Data/tumor1.nrrd")
# run_window("Data/tumor1.nrrd")

# 4. Analyse et visualisation des changements

In [289]:
100 * np.count_nonzero(boxed_tumor_slices) / np.count_nonzero(original_slices) - 100

13.141133087495803

In [290]:
100 * int(boxed_tumor_slices.sum(axis=2).sum()) / int(original_slices.sum(axis=2).sum()) - 100

10.88816796072345

L'intensité des voxels de la tumeur n'a pas augmenté autant que leur nombre, ce qui tend à indiquer que certaines parties de la tumeur ont une concentration en cellules cancéreuses moins élevée. Cette différence pourrait cependant aussi être attribuée à des erreurs d'approximation provenant des processus de recalage et de segmentation.

In [291]:
r = original_slices
g = np.zeros(shape=original_slices.shape)
b = boxed_tumor_slices

rgb = np.stack([r, g, b], axis=3)

@interact(slice=(70, original_slices.shape[0] - 1))
def display(slice: int = 82):
    onion_skin = rgb[slice, *box_slice[1:]]
    if (box_max := np.max(onion_skin.flat)) != 0:
        onion_skin /= box_max

    plt.figure(figsize=(15, 10))

    plt.subplot(2, 2, 1)
    plt.title("Original tumor")
    plt.imshow(r[slice, *box_slice[1:]])

    plt.subplot(2, 2, 2)
    plt.title("Evolved tumor")
    plt.imshow(b[slice, *box_slice[1:]])

    plt.subplot(2, 2, 3)
    plt.title("Onion skinning")
    plt.imshow(onion_skin)

    plt.subplot(2, 2, 4)
    plt.title("Difference (evolved - original)")
    plt.imshow(b[slice, *box_slice[1:]] - r[slice, *box_slice[1:]], cmap='gray')


interactive(children=(IntSlider(value=82, description='slice', max=86, min=70), Output()), _dom_classes=('widg…

On peut confirmer ici visuellement que les augmentations relevées semblent justes.